In [3]:
from IPython.display import clear_output
from tabulate import tabulate
from deepspeed.profiling.flops_profiler import get_model_profile
import utils, importlib
from models import LinearReLU, LinearBSpline
import linspline
from sklearn.datasets import fetch_california_housing
import time, torch
from torch.utils.data import TensorDataset, DataLoader

import importlib, utils

importlib.reload(linspline)

archs_layers = [

    #^ Testing number of control points:

    # ("BSpline", [8]),
    # ("BSpline", [24, 8]),
    # ("BSpline", [24, 48, 24, 8]),
    # ("BSpline", [24, 48, 96, 48, 24, 8]),

    #^ Comparing across architectures

    # ("ReLU", [8]),
    # ("BSpline", [8]),
    ("LSpline", [8]),

    # ("ReLU", [24, 8]),
    # ("BSpline", [24, 8]),
    # ("LSpline", [24, 8]),

    # ("ReLU", [24, 48, 24, 8]),
    # ("BSpline", [24, 48, 24, 8]),
    # ("LSpline", [24, 48, 24, 8]),

    # ("ReLU", [8, 48, 192, 48, 8]),
    # ("ReLU", [24, 48, 96, 24, 8]),
    # ("BSpline", [24, 48, 96, 24, 8]),
    # ("LSpline", [24, 48, 96, 24, 8]),

    #^ Controlling for parameters:
    # ("BSpline", [8]),
    # ("ReLU", [11]),

    # ("BSpline", [24, 8]),
    # ("ReLU", [30, 8]),

    # ("BSpline", [24, 48, 24, 8]),
    # ("ReLU", [26, 50, 26, 8]),

    # ("BSpline", [24, 48, 96, 48, 24, 8]),
    # ("ReLU", [26, 50, 98, 50, 26, 8]),

    #^ Controlling for FLOPS

    # ("BSpline", [8]),
    # ("ReLU", [24]),

    # ("BSpline", [24, 8]),
    # ("ReLU", [64, 8]),

    # ("BSpline", [24, 48, 24, 8]),
    # ("ReLU", [24, 96, 24, 8]),

    # ("BSpline", [24, 48, 96, 48, 24, 8]),
    # ("ReLU", [24, 48, 144, 48, 24, 8]),

    #^ Controlling for fwd lat

    # ("BSpline", [8]),
    # ("ReLU", [16, 32, 64, 128, 64, 32, 16, 8]),

    # ("BSpline", [16, 8]),
    # ("ReLU", [24, 48, 96, 192, 384, 576, 384, 192, 96, 48, 24, 8]),

    # ("BSpline", [24, 48, 24, 8]),
    # ("ReLU", [24, 48, 192, 768, 1152, 2304, 1152, 768, 192, 48, 24, 8]),

]

store = []

# for ctrl in [0,3,5,11,23,55,111]:
ctrl=3
for (arch, layers) in archs_layers:
    if(ctrl == 0):
        arch="ReLU"
    try:
        flops, params, fwd_lat_real = utils.profile_model(arch, layers, ctrl=ctrl)
        store.append([f"{arch} {layers} ctrl={ctrl}", params, flops, fwd_lat_real])
    except Exception as e:
        print("Error: ", e, "on ", (arch, layers))
        store.append([f"{arch} {layers}", f"{e}", 0, 0])
        
    clear_output()

headers = ["Model", "Params", "FLOPs/input", "μs/input"]
print(tabulate(store, headers=headers, tablefmt="grid"))

+--------------------------------+----------+---------------+------------+
| Model                          | Params   |   FLOPs/input |   μs/input |
+================================+==========+===============+============+
| ReLU [8] ctrl=3                | 81       |           152 |       3.61 |
+--------------------------------+----------+---------------+------------+
| LSpline [8] ctrl=3             | 105      |           312 |      12.29 |
+--------------------------------+----------+---------------+------------+
| ReLU [24, 8] ctrl=3            | 425      |           816 |       8.85 |
+--------------------------------+----------+---------------+------------+
| LSpline [24, 8] ctrl=3         | 521      |          1456 |      27.74 |
+--------------------------------+----------+---------------+------------+
| ReLU [24, 48, 24, 8] ctrl=3    | 2.8 K    |          5496 |      15.1  |
+--------------------------------+----------+---------------+------------+
| LSpline [24, 48, 24, 8]

In [13]:
from IPython.display import clear_output
from tabulate import tabulate
from deepspeed.profiling.flops_profiler import get_model_profile
import utils, importlib
from models import LinearReLU, LinearBSpline
import linspline
from sklearn.datasets import fetch_california_housing
import time, torch
from torch.utils.data import TensorDataset, DataLoader
import importlib, utils

importlib.reload(linspline)
importlib.reload(utils)


archs_layers = [
    ("LSpline", [7]),
    ("LSpline", [6]),
    ("LSpline", [5]),
    ("LSpline", [4]),
]

store = []

ctrl=3
for (arch, layers) in archs_layers:
    if(ctrl == 0):
        arch="ReLU"
    try:
        flops, params, fwd_lat_real = utils.profile_model(arch, layers, ctrl=ctrl)
        store.append([f"{arch} {layers} ctrl={ctrl}", params, flops, fwd_lat_real])
    except Exception as e:
        print("Error: ", e, "on ", (arch, layers))
        store.append([f"{arch} {layers}", f"{e}", 0, 0])
        
    clear_output()

headers = ["Model", "Params", "FLOPs/input", "μs/input"]
print(tabulate(store, headers=headers, tablefmt="grid"))

+--------------------+----------+---------------+------------+
| Model              |   Params |   FLOPs/input |   μs/input |
+====================+==========+===============+============+
| LSpline [7] ctrl=3 |       92 |           273 |      12.15 |
+--------------------+----------+---------------+------------+
| LSpline [6] ctrl=3 |       79 |           234 |      12.17 |
+--------------------+----------+---------------+------------+
| LSpline [5] ctrl=3 |       66 |           195 |      12.32 |
+--------------------+----------+---------------+------------+
| LSpline [4] ctrl=3 |       53 |           156 |      12.18 |
+--------------------+----------+---------------+------------+
